### Remove and recreate output folder

In [ ]:
import os
import shutil

outdir = './output'

if os.path.exists(outdir):
    shutil.rmtree(outdir)
    
if not os.path.exists(outdir):
    os.mkdir(outdir)

### Discovery of Main Jobs and Job Performers

In [ ]:
message = """

"""

additional_prompt = ""

In [ ]:
from importlib import reload
import autogen

import lib.autogen.agents as this
reload(this)


groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.job_developer,
        this.critic_agent,
    ],
    messages=[],
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=message
)

In [ ]:
import re
import pandas as pd

filtered_items = [item for item in res.chat_history if item.get('name') == 'Job_Developer']

pattern = r'```\w*\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)

with open("./output/performers_and_jobs.csv", 'w') as file:
    file.write(matches[0])

performers_and_jobs = pd.read_csv("./output/performers_and_jobs.csv", delimiter=';', index_col=False)

performers_and_jobs


In [ ]:
import pandas as pd

performers_and_jobs = pd.read_csv("./output/performers_and_jobs.csv", delimiter=';', index_col=False)

try:
    performers_and_jobs = performers_and_jobs.rename(columns={'Role': 'Job Performer'})
except:
    pass

all_performers_and_jobs = pd.read_csv("./output/all_performers_and_jobs.csv", delimiter=';', index_col=False)

all_performers_and_jobs = pd.concat([all_performers_and_jobs, performers_and_jobs], ignore_index=True)
all_performers_and_jobs = all_performers_and_jobs.sort_values(by=all_performers_and_jobs.columns[0])
all_performers_and_jobs.to_csv("./output/all_performers_and_jobs.csv", sep=';', index=False)

all_performers_and_jobs

In [ ]:
all_performers_and_jobs

### Rate the different possible markets (aka Job_Performers + jobs) using Disciplined Enterpreneur book

In [ ]:
import pandas as pd

performers_and_jobs = pd.read_csv("./output/all_performers_and_jobs.csv", delimiter=';', index_col=False)

message = f"""
Here the Job Performers list:
{';'.join(list(performers_and_jobs['Job Performer'].unique()))}
"""

In [ ]:
from importlib import reload
import autogen

import lib.autogen.agents as this
reload(this)


groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.business_expert,
        this.critic_agent,
    ],
    messages=[],
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=message
)

In [ ]:
import re

filtered_items = [item for item in res.chat_history if item.get('name') == 'Business_Expert']

pattern = r'```\w*\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)

with open("./output/performers_scores.csv", 'w') as file:
    file.write(matches[0])

performers_scores = pd.read_csv("./output/performers_scores.csv", delimiter=';', index_col=False)

performers_scores.sort_values('Total Score',ascending=False)


### Synthesis of Job Performers

In [ ]:
selected_job_performers = ""
selected_main_job = ""
number = "5"
additional_prompt   = ""

In [ ]:
from importlib import reload
import autogen

import lib.autogen.agents as this
reload(this)

groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.performer_generator,
        this.critic_agent,
    ], 
    messages=[], 
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=f"""

Generate {number}

Job Performers: {selected_job_performers}
In which their Main Job is {selected_main_job}
"""
)


In [ ]:
import re

filtered_items = [item for item in res.chat_history if item.get('name') == 'Performer_generator']

pattern = r'```\w*\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)

with open("./output/performers.csv", 'w') as file:
    file.write(matches[0])

performers = pd.read_csv("./output/performers.csv", delimiter=';', index_col=False)

performers

### Automated Interviews

In [ ]:
import pandas as pd

performers = pd.read_csv('./output/performers.csv',delimiter=';', index_col=False)

In [ ]:
import json

from importlib import reload

import lib.interview_performers as this
reload(this)

interviews = []
for index, row in performers.iterrows():
    performer = row.to_json()

    main_job_related = f"""Answer the following question focusing and thinking about the main job {selected_main_job} applied to what you are doing as {row['Profession']}.
    If the main job {selected_main_job} is not related to you, simply answer \"This is not my main job\" and skip the question."""

    # main_job_related = ""

    res = this.chain.invoke({"job_performer": performer, "main_job_related": main_job_related})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])
    
    interview = pd.DataFrame(res_dict["interview"])
    
    job_performer_dict = row.to_dict()

    job_performer_dict.update({"interview": interview})

    interviews.append(job_performer_dict)

In [ ]:
import os
import json
import copy

outdir = './output/interviews'
if not os.path.exists(outdir):
    os.mkdir(outdir)

interviews_to_save = copy.deepcopy(interviews)

for interview in interviews_to_save:
    filename = os.path.join(outdir,"_".join((
        interview["Name"].replace(" ", ""),
        interview["Profession"].replace(" ", ""),
        ))+".json")

    interview["interview"] = interview["interview"].to_dict(orient='records') 

    with open(filename, 'w') as json_file:
        json.dump(interview, json_file)

del interviews_to_save

### Analyze Interviews

In [ ]:
import os
import json

outdir = './output/interviews'

# Initialize an empty list to store the data
interviews_to_analyze = []

# Loop through each file in the folder
for filename in os.listdir(outdir):
    if filename.endswith('.json'):
        file_path = os.path.join(outdir, filename)
        with open(file_path, 'r') as file:
            # Load the JSON data from the file
            json_data = json.load(file)
            # Append the data to the list
            interviews_to_analyze.append(json_data)

In [ ]:
from importlib import reload
import autogen

import lib.autogen.agents as this
reload(this)

groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.jtbd_analyzer,
        this.critic_agent,
    ], 
    messages=[], 
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=f"""
Analyse the following interviews using JTBD framework
{interviews_to_analyze}
"""
)

In [ ]:
import re
import pandas as pd

filtered_items = [item for item in res.chat_history if item.get('name') == 'JTBD_Analyzer']

pattern = r'```\w*\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)

with open("./output/analysis.csv", 'w') as file:
    file.write(matches[0])

analysis = pd.read_csv("./output/analysis.csv", delimiter=';', index_col=False)

analysis

### Categorize Interviews

In [ ]:
import pandas as pd

analysis = pd.read_csv("./output/analysis.csv", delimiter=';', index_col=False)

In [ ]:
from importlib import reload
import autogen

import lib.autogen.agents as this

reload(this)

groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.critic_agent,
        this.jtbd_categorizer,
    ], 
    messages=[], 
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=f"""

The main job is {selected_main_job} while the steps are:

{analysis}

"""
)

In [ ]:
import re

filtered_items = [item for item in res.chat_history if item.get('name') == 'JTBD_Categorizer']

pattern = r'```\w*\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)

with open("./output/category.csv", 'w') as file:
    file.write(matches[0])

category = pd.read_csv("./output/category.csv", delimiter=';', index_col=False)

custom_order = [
    'Define',
    'Locate',
    'Prepare',
    'Confirm',
    'Execute',
    'Monitor',
    'Modify',
    'Conclude'
]

category['Category'] = pd.Categorical(category['Category'], categories=custom_order, ordered=True)
category_sorted = category.sort_values('Category').reset_index(drop=True)

category_sorted.to_csv("./output/category.csv", sep=';')

### Find Solutions

In [ ]:
import pandas as pd

category_sorted = pd.read_csv("./output/category.csv", delimiter=';', index_col=False)
category_sorted


In [ ]:

from importlib import reload
import autogen

import lib.autogen.agents as this

reload(this)

groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.planner,
        this.solution_generator,
        this.digital_product_designer,
        this.critic_agent,
    ], 
    messages=[], 
    max_round=10
)


manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=f"""

We run a Jobs-to-be-done Analysis. Here the information:

Job Performer: {selected_job_performers}

Main Jobs: {selected_main_job}

Job Steps and Needs:
{category_sorted[["Job Step", "Need"]]}
"""
)

In [ ]:
import re
import json

filtered_items = [item for item in res.chat_history if item.get('name') == 'Digital_Product_Designer']

pattern = r'```json\n(.*?)```'
matches = re.findall(pattern, filtered_items[-1]['content'], re.DOTALL)


solutions = json.loads(matches[0])[0]

with open("./output/solutions.json", 'w') as file:
    json.dump(solutions, file, indent=4)


### Create Narrative for Landing Pages

In [ ]:
import json

with open("./output/solutions.json") as file:
    solutions = json.load(file)

In [ ]:

from importlib import reload
import autogen

import lib.autogen.agents as this

reload(this)

groupchat = autogen.GroupChat(
    agents=[
        this.user,
        this.storybrand_expert,
        this.critic_agent,
        this.copywriting_expert
    ], 
    messages=[], 
    max_round=10
)


manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=this.gpt4_config)

res = this.user.initiate_chat(
    manager,
    message=f"""

Here the document describing the solutions and the problem that comes from JTBD.

{solutions}
"""
)

